In [9]:
import numpy as np
import re

# Inverse kinematic

## Symbol calculations

In [25]:
import sympy
sympy.init_printing()

In [26]:
Rx, Ry = sympy.symbols("R_x, R_y")
rx, ry = sympy.symbols("r_x, r_y")
vx, vy, w = sympy.symbols("v_x, v_y, \omega")

In [27]:
V = sympy.Matrix([vx, vy, 0]) + sympy.Matrix([0, 0, w]).cross(sympy.Matrix([Rx, Ry, 0]))
V_m = sympy.Matrix(V[0:2]).dot(sympy.Matrix([rx, ry]))
V_m = sympy.expand(V_m)

In [28]:
V_m

## Numerical calculations

In [158]:
Rx1 = 175.27
Ry1 = 120.27 / 2
a = np.pi / 4
rwheel = 28.8

rx1 = np.cos(a)
ry1 = np.sin(a)
angles = [np.pi / 4,  65 / 180 * np.pi, ]
Rxs = [Rx1, Rx1, -Rx1, -Rx1]
Rys = [Ry1, -Ry1, Ry1, -Ry1]
rxs = [rx1, -rx1, rx1, -rx1]
rys = [-ry1, -ry1, ry1, ry1]

In [159]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [160]:
forward_matrix * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.55231857, -24.55231857,  -5.77973843],
       [ 24.55231857,  24.55231857,   5.77973843],
       [ 24.55231857,  24.55231857,  -5.77973843],
       [ 24.55231857, -24.55231857,   5.77973843]])

### Forward kinematic

In [161]:
np.linalg.lstsq(forward_matrix, np.eye(4), rcond=None)

(array([[ 0.01018234, -0.01018234,  0.01018234, -0.01018234],
        [-0.01018234, -0.01018234,  0.01018234,  0.01018234],
        [-0.04325455, -0.04325455, -0.04325455, -0.04325455]]),
 array([0.25, 0.25, 0.25, 0.25]),
 3,
 array([49.10463715, 49.10463715, 11.55947685]))

## Different angles for big robot center in geom center

<img src="files/photo_big_robot_2.jpg">

In [162]:
Rx1 = 175.82
Rx2 = 172.15
Ry1 = 60.82
Ry2 = 62.84

Rxs = [-Rx1, -Rx2, Rx2, Rx1]
Rys = [Ry1, -Ry2, -Ry2, Ry1]
alphas = (np.array([180 - 45, 180 + 65, -65, 45]) - 90) * np.pi / 180
rxs = np.cos(alphas)
rys = np.sin(alphas)

In [163]:
rxs

array([ 0.70710678, -0.90630779, -0.90630779,  0.70710678])

In [164]:
rys

array([ 0.70710678,  0.42261826, -0.42261826, -0.70710678])

In [165]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [166]:
forward_matrix * np.array([1, -1, -1, 1])[:, np.newaxis]

array([[ 24.55231857,  24.55231857,  -5.8100605 ],
       [ 31.46902084, -14.67424583,   4.50368452],
       [ 31.46902084,  14.67424583,   4.50368452],
       [ 24.55231857, -24.55231857,  -5.8100605 ]])

In [167]:
np.linalg.lstsq(forward_matrix, np.eye(4), rcond=None)

(array([[ 0.00767466, -0.00990083, -0.00990083,  0.00767466],
        [ 0.01500478,  0.00896795, -0.00896795, -0.01500478],
        [-0.05362585, -0.04183921, -0.04183921, -0.05362585]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([56.44672134, 40.45120117, 10.3960976 ]))

# Calibration

### New big robot

In [168]:
y_exp = 890
y_true = 890 - 83 
x_exp = 1719
x_true = 1719 + 36
a_exp = np.pi
a_true = np.pi - 1 / 40

In [169]:
k = np.array([x_exp / x_true, y_exp / y_true, a_exp / a_true])

In [170]:
forward_matrix_1 = forward_matrix * k[np.newaxis, :]

In [171]:
forward_matrix_1 * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.04868127,  27.07752606,  -5.85666637],
       [ 30.82350247, -16.18349293,   4.53981119],
       [-30.82350247, -16.18349293,  -4.53981119],
       [-24.04868127,  27.07752606,   5.85666637]])

In [172]:
np.linalg.lstsq(forward_matrix_1, np.eye(4), rcond=None)

(array([[ 0.00783538, -0.01010818, -0.01010818,  0.00783538],
        [ 0.01360546,  0.00813161, -0.00813161, -0.01360546],
        [-0.05319911, -0.04150626, -0.04150626, -0.05319911]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([55.28884046, 44.61160972, 10.47949063]))

# Calibration data

In [173]:
def cvt_local2global(local_point, sc_point):
    point = np.zeros(3)
    x, y, a = local_point
    X, Y, A = sc_point
    point[0] = x * np.cos(A) - y * np.sin(A) + X
    point[1] = x * np.sin(A) + y * np.cos(A) + Y
    point[2] = a + A
    return point


def cvt_global2local(global_point, sc_point):
    point = np.zeros(3)
    x, y, a = global_point
    X, Y, A = sc_point
    point[0] = x * np.cos(A) + y * np.sin(A) - X * np.cos(A) - Y * np.sin(A)
    point[1] = -x * np.sin(A) + y * np.cos(A) + X * np.sin(A) - Y * np.cos(A)
    point[2] = a - A
    return point

def get_movement(point):
    x, y, a = point
    R = np.sqrt(x ** 2 + y ** 2) / 2 / np.abs(np.sin(a / 2))
    S = R * a
    angle = np.arctan2(y, x) - a / 2
    return np.array([S * np.cos(angle), S * np.sin(angle), a])

In [174]:
import re

In [175]:
points = []
cmds = []
with open("calibration_data.txt", 'r') as file:
    for line in file:
        m = re.match('- Translation: \[(.*)\]', line[:-1]) 
        if m:        
            points.append(np.zeros(3))
            points[-1][0:2] = np.fromstring(m.group(1), sep=', ')[0:2]
        m = re.match('[ ]*in RPY \(radian\) \[(.*)\]', line[:-1]) 
        if m:
            points[-1][2] = np.fromstring(m.group(1), sep=', ')[2]
        m = re.match('(-?[\d]+.?[\d]* -?[\d]+.?[\d]* -?[\d]+.?[\d]*)', line[:-1])
        if m:
            cmds.append(np.fromstring(m.group(1), sep=' ', dtype=np.float32))

points_1 = np.array(points)
cmds = np.array(cmds)

In [176]:
points = np.array([cvt_global2local([0, 0, 0], p) for p in points_1])
true_movements = np.array([get_movement(cvt_global2local(p2, p1)) for p1, p2 in zip(
    points[:-1], np.roll(points, -1, axis=0)[:-1])])

In [185]:
true_movements.round(2)
for i in range(len(cmds) - 1):
    print((cmds[i] - true_movements[i]).round(2))

[ 0.02 -0.03 -0.02]
[ 0.01 -0.   -0.03]
[ 0.1  -1.68 -3.07]
[-1.63 -0.06  0.05]
[ 0.    0.01 -0.02]
[0.02 1.9  6.04]
[-0.04 -0.04 -0.49]
[ 0.04 -0.03 -0.41]
[-1.99 -0.01  0.02]
[ 0.02  0.03 -0.36]
[0.45 0.02 5.72]
[-0.01 -0.01 -0.57]
[ 0.02 -2.01  0.02]
[-0.8  -0.    0.01]
[-0.    0.    0.01]
[-0.  0. -0.]
[ 0. -0.  0.]
[-0.1   1.01 -0.05]
[0.12 0.02 0.01]
[ 0.01  0.01 -0.02]


In [186]:
errors = cmds[:-1] - true_movements
conds = (np.abs(errors[:, 0]) < 0.1) * (np.abs(errors[:, 1]) < 0.1) *\
    (np.abs(errors[:, 2]) < 0.05)
X = true_movements[conds]
Y = cmds[:-1][conds]

In [187]:
K = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y).round(2)
K

array([[ 1.01, -0.  , -0.02],
       [ 0.03,  0.97, -0.06],
       [-0.  ,  0.01,  0.99]])

In [188]:
forward_matrix2 = forward_matrix_1.dot(K)

In [189]:
forward_matrix2 * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 25.10149386,  26.20663361,  -7.9037249 ],
       [ 30.6462327 , -15.65259003,   4.8489526 ],
       [-31.61724228, -15.74338625,  -2.90693345],
       [-23.4768423 ,  26.32376694,   4.65442177]])

In [190]:
np.linalg.lstsq(forward_matrix2, np.eye(4), rcond=None)

(array([[ 0.00669162, -0.01083953, -0.01083618,  0.00669722],
        [ 0.01048883,  0.00612119, -0.01063465, -0.01754632],
        [-0.05384242, -0.04198735, -0.0418181 , -0.05355923]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([55.89738448, 43.32378132, 10.35868412]))